# Settings

In [1]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
\


In [2]:
import tensorflow as tf
print(tf.__version__)

KeyboardInterrupt: 

# Dataset loading

In [ ]:
dataset_name='pokemon'

In [ ]:
IMG_DIR = 'C:\\projects\\pokemon\DS06\\'
VAL_PCT = 10
VAL_FORMAT = 'png'

In [ ]:
from training.generators.file_image_generator import create_image_lists, get_generators

In [ ]:
imgs_list = create_image_lists(
    image_dir=IMG_DIR, 
    validation_pct=VAL_PCT, 
    valid_imgae_formats=VAL_FORMAT
)

In [ ]:
inputs_shape= IMG_SIZE=(64, 64, 3)
BATCH_SIZE = 32
latent_dim = 100
intermediate_dim = 100
TRAIN_BUF = 600
TEST_BUF = 100

In [ ]:
train_gen, test_gen = get_generators(
    images_list=imgs_list, 
    image_dir=IMG_DIR, 
    image_size=IMG_SIZE, 
    batch_size=BATCH_SIZE, 
    class_mode=None
)

In [ ]:
import tensorflow as tf

In [ ]:
train_ds = tf.data.Dataset.from_generator(
    lambda: train_gen, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((BATCH_SIZE, ) + IMG_SIZE)
)


In [ ]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [ ]:
_instance_scale

In [ ]:
import numpy as np
from collections.abc import Iterable

In [ ]:
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [ ]:
_outputs_shape

# Model's Layers definition

In [ ]:
enc_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latent_dim)]

dec_lays = [tf.keras.layers.Dense(units=latent_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)]

# Model definition

In [ ]:
model_name = dataset_name+'AEpsnr'
recoding_dir='..'+sep_local+'..'+sep_local+'recoding'+sep_local+model_name

In [ ]:
from training.traditional.autoencoders.autoencoder import autoencoder as AE

In [ ]:
inputs_shape=IMG_SIZE

In [ ]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latent_dim,
        'layers': enc_lays#enc_lays2#
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latent_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays#dec_lays2#
    }
]

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist

In [ ]:
_restore = os.path.join(recoding_dir, 'var_save_dir')

In [ ]:
create_if_not_exist(_restore)
_restore

In [ ]:
#to restore trained model, set filepath=_restore

In [ ]:
ae = AE( 
    name=model_name,
    inputs_shape=inputs_shape,
    outputs_shape=inputs_shape,
    latent_dim=latent_dim,
    batch_size=BATCH_SIZE,
    variables_params=variables_params, 
    filepath=None
    )

In [ ]:
from evaluation.quantitive_metrics.psnr import psnr
#from evaluation.quantitive_metrics.ssmi import ssmi
#from evaluation.quantitive_metrics.sharp_diff import sharp_diff


In [ ]:
ae.compile(loss={'x_logits': psnr})

# Callbacks

In [ ]:
from training.callbacks.progress_bar import NotebookPrograssBar
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [ ]:
progbar = NotebookPrograssBar(leave_outer=False)

In [ ]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [ ]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [ ]:
csv_dir = os.path.join(recoding_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

In [ ]:
image_gen_dir = os.path.join(recoding_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [ ]:
sg = SampleGeneration(latent_shape=latent_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

In [ ]:
import numpy as np

# Model Training

In [ ]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=50,
    epochs=int(1e6), 
    verbose=0,
    callbacks=[progbar, es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True
)